***
# Импорт библиотек
* импорт библиотек
* загрузка констант из файла `settings.yaml`

In [1]:
from src.utils import *
globals().update(load_settings(True))

Loaded following classes:
['DataLoader', 'Dataset', 'MainRecommender']

Loaded following functions:
['load_settings', 'pprint', 'precision_at_k']

Loaded following constants:
ITEM_COL             = item_id              | <class 'str'>
USER_COL             = user_id              | <class 'str'>
ACTUAL_COL           = actual               | <class 'str'>
TAKE_N_POPULAR       =                    5 | <class 'int'>
N_PREDICT            =                   50 | <class 'int'>
TOPK_PRECISION       =                    5 | <class 'int'>
VAL_MATCHER_WEEKS    =                    6 | <class 'int'>
VAL_RANKER_WEEKS     =                    3 | <class 'int'>
CATBOOST_RANKER      = YetiRankPairwise     | <class 'str'>
CATBOOST_ITERATIONS  =                 2000 | <class 'int'>


***
# Предоброботка данных
* загрузка датасетов
* первичная трансформация датасетов
* добавление новых фич
* конвертация текстовых фичей в цифровые для модели второго уровня
* разделение и подготовка датасетов для формирования рекомендаций и последующего ранжирования

In [2]:
data = Dataset()
data.item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [3]:
data.data_prefilter()
data.user_features.head()

,household_size_desc,kid_category_desc,user_id,age,income,has_kids,Homeowner,Probable Owner,Probable Renter,Renter,Unknown,A,B,U
0,2,0,1,75.0,42.0,0.0,1,0,0,0,0,1,0,0
1,2,0,7,49.0,62.0,0.0,1,0,0,0,0,1,0,0
2,3,1,8,29.0,29.0,1.0,0,0,0,0,1,0,0,1
3,4,2,13,29.0,87.0,1.0,1,0,0,0,0,0,0,1
4,1,0,16,49.0,62.0,0.0,1,0,0,0,0,0,1,0


In [4]:
data.data_split()

In [5]:
data.data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,index,avg_check,quantity_total
0,1364,26984896261,1,842930,1,2.19,31742,0.00,1520,1,0.0,0.0,440,2.726185,125
1,1364,26984896261,1,897044,1,2.99,31742,-0.40,1520,1,0.0,0.0,440,2.726185,27
3,1364,26984896261,1,937406,1,2.50,31742,-0.99,1520,1,0.0,0.0,440,2.726185,47
4,1364,26984896261,1,981760,1,0.60,31742,-0.79,1520,1,0.0,0.0,440,2.726185,5778
5,1364,27081336145,9,915041,1,0.99,31742,-0.40,1554,2,0.0,0.0,440,2.726185,208


In [6]:
data.data_test.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,index,avg_check,quantity_total
0,588,41652838477,664,1024426,1,6.29,388,0.00,8,96,0.0,0.0,197,2.885,1
1,588,41678913231,665,6534178,17885,34.50,388,-1.79,1933,96,0.0,0.0,197,2.885,5096037
2,588,41717371824,668,9673270,1,15.00,388,0.00,1514,96,0.0,0.0,197,2.885,1
3,588,41756247555,671,826842,1,1.99,388,0.00,1738,97,0.0,0.0,197,2.885,2
4,588,41756247555,671,833025,1,1.00,388,-0.16,1738,97,0.0,0.0,197,2.885,67


***
# Формирование рекомендаций <a class="anchor" id="third-bullet"></a>
* train_test_split
* получение рекомендаций по собственным покупкам (с добавлением популярных товаров)

In [7]:
recommender = MainRecommender(data.data_train_lvl_1)
recommender.preprocessing(data, t='train')
recommender.preprocessing(data, t='test')

100%|█████████████████████████████████████████| 15.0/15 [00:10<00:00,  1.37it/s]
100%|██████████████████████████████████| 48263/48263 [00:02<00:00, 20113.55it/s]


In [8]:
recommender.recommend_1lvl(data.result_lvl_1).head(1)

,user_id,actual,own_recs,own_recs_score
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[1049998, 1087895, 949257, 1032924, 6904485, 1...",0.132743


***
# Ранжирование полученных рекомендаций <a class="anchor" id="forth-bullet"></a>
* форматируем датасеты под формат (class Pool) CatBoost
* обучаем CatBoostRanker с функцией потерь YetiRankPairwise
* получаем предсказанные вероятности для ранжирования
* ранжируем рекомендации от модели первого уровня

In [9]:
recommender.ranker_fit()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [10]:
recommender.ranker_predict(recommender.X_train).head()

,user_id,item_id,manufacturer,brand,commodity_type,household_size_desc,kid_category_desc,age,income,has_kids,Homeowner,Probable Owner,Probable Renter,Renter,Unknown,A,B,U,predict
0,1364,1014831,869,0,84,1,0,75.0,112.0,0.0,1,0,0,0,0,0,1,0,0.602435
1,1364,7169090,2110,0,9,1,0,75.0,112.0,0.0,1,0,0,0,0,0,1,0,0.409390
2,1364,1101378,2224,0,63,1,0,75.0,112.0,0.0,1,0,0,0,0,0,1,0,0.470725
3,1364,1090252,2224,0,63,1,0,75.0,112.0,0.0,1,0,0,0,0,0,1,0,0.470725
4,1364,7152319,2627,0,201,1,0,75.0,112.0,0.0,1,0,0,0,0,0,1,0,0.360282


***
# Оценка полученных результатов

In [11]:
recommender.evaluate_2models()

precision@5 of 2lvl-model is 0.09712041884816791


***
# Сохранение результата <a class="anchor" id="sixth-bullet"></a>
* повторяем все вышеописанные действия для "боевой" базы (обучаемся на трейн_сете, предсказываем по тест_сету)
* сохраняем результат в .xml

In [12]:
data.data_test_split()
recommender = MainRecommender(data.data_train_lvl_1_real)
recommender.preprocessing(data, t='train', training=False)
recommender.preprocessing(data, t='test', training=False)
recommender.ranker_fit(training=False)
recommender.evaluate_2models(training=False)

100%|█████████████████████████████████████████| 15.0/15 [00:10<00:00,  1.47it/s]
100%|██████████████████████████████████| 52244/52244 [00:03<00:00, 14569.45it/s]


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

precision@5 of 2lvl-model is 0.09732620320855655
Файл с результатами сохранен.


***